In [ ]:
! pip install -qU memorizz yahooquery

In [2]:
import getpass
import os

# Function to securely get and set environment variables
def set_env_securely(var_name, prompt):
    value = getpass.getpass(prompt)
    os.environ[var_name] = value

In [4]:
set_env_securely("MONGODB_URI", "Enter your MongoDB URI: ")

In [5]:
set_env_securely("OPENAI_API_KEY", "Enter your OpenAI API Key: ")

In [6]:
from memorizz.memory_provider.mongodb.provider import MongoDBConfig, MongoDBProvider

# Create a memory provider
mongodb_config = MongoDBConfig(uri=os.environ["MONGODB_URI"])
memory_provider = MongoDBProvider(mongodb_config)

In [7]:
from memorizz.memory_component.memory_mode import MemoryMode
from memorizz.workflow.workflow import Workflow
from memorizz import Toolbox
import requests

def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

def calculate_distance(city1: str, city2: str) -> str:
    """Calculate distance between two cities."""
    # Simulate distance calculation
    distances = {
        ("New York", "London"): "3,459 miles (5,567 km)",
        ("New York", "Tokyo"): "6,743 miles (10,850 km)",
        ("London", "Paris"): "214 miles (344 km)",
        ("Tokyo", "Sydney"): "4,840 miles (7,788 km)",
        ("New York", "Paris"): "3,631 miles (5,844 km)"
    }
    key = (city1, city2) if (city1, city2) in distances else (city2, city1)
    return distances.get(key, f"Distance data not available between {city1} and {city2}")

def book_flight(origin: str, destination: str, date: str) -> str:
    """Book a flight between two cities."""
    # Simulate flight booking (will sometimes "fail" for demonstration)
    if "Mars" in origin or "Mars" in destination:
        raise Exception("No flights available to Mars at this time")
    
    price = hash(f"{origin}{destination}{date}") % 500 + 200
    return f"Flight booked from {origin} to {destination} on {date}. Price: ${price}. Confirmation: FL{hash(date) % 10000}"

def get_currency_rate(from_currency: str, to_currency: str) -> str:
    """Get currency exchange rate."""
    # Simulate currency rates
    rates = {
        ("USD", "EUR"): "0.85",
        ("USD", "GBP"): "0.73",
        ("USD", "JPY"): "110.0",
        ("EUR", "USD"): "1.18",
        ("GBP", "USD"): "1.37"
    }
    key = (from_currency.upper(), to_currency.upper())
    rate = rates.get(key, "1.00")
    return f"1 {from_currency.upper()} = {rate} {to_currency.upper()}"

In [ ]:
# Create toolbox and register tools, with memory provider
toolbox = Toolbox(memory_provider=memory_provider)

toolbox.register_tool(get_weather)
toolbox.register_tool(calculate_distance)
toolbox.register_tool(book_flight)
toolbox.register_tool(get_currency_rate)


In [9]:
def print_section(title):
    """Print a formatted section header."""
    print(f"\n{'='*60}")
    print(f" {title}")
    print(f"{'='*60}")

In [10]:
def print_workflow_info(workflow):
    """Print workflow information in a readable format."""
    print(f"Workflow: {workflow.name}")
    print(f"Description: {workflow.description}")
    print(f"Outcome: {workflow.outcome.value}")
    print(f"Created: {workflow.created_at}")
    print(f"Steps: {len(workflow.steps)}")
    
    for step_name, step_data in workflow.steps.items():
        print(f"  - {step_name}")
        print(f"    Function: {step_data.get('_id', 'Unknown')}")
        print(f"    Arguments: {step_data.get('arguments', {})}")
        if step_data.get('error'):
            print(f"    Error: {step_data.get('error')}")
        else:
            result = step_data.get('result', 'No result')
            # Handle different result types properly
            if isinstance(result, str):
                print(f"    Result: {result[:100]}..." if len(result) > 100 else f"    Result: {result}")
            else:
                # For non-string results (like float, int, etc.), convert to string first
                result_str = str(result)
                print(f"    Result: {result_str[:100]}..." if len(result_str) > 100 else f"    Result: {result_str}")

In [11]:
import time
from memorizz.memagent import MemAgent

def demonstrate_workflow_memory():
    """Main demonstration function."""
    
    print_section("Workflow Memory Demonstration")
    print("This demo shows how MemAgent learns from previous tool executions")
    print("and uses that knowledge to guide future similar tasks.\n")
    
    # Create memory provider and toolbox    
    # Create agent with workflow memory mode
    print("Creating MemAgent with Workflow memory mode...")
    agent = MemAgent(
        tools=toolbox,
        memory_mode=MemoryMode.Workflow, # Set the memory mode to Workflow
        memory_provider=memory_provider, # Add the memory provider
        instruction="You are a helpful travel assistant. Use the available tools to help users with travel-related queries."
    )

    # agent.add_tool(toolbox=toolbox)

    print("Created agent with tools: ", agent.tools)
    
    # Save the agent
    agent.save()
    
    # Phase 1: Create initial workflows
    print_section("Phase 1: Creating Initial Workflows")
    
    # Successful workflow example
    print("\n1. Running travel planning query (will create successful workflow)...")
    response1 = agent.run("I want to plan a trip from New York to London. Can you help me with weather and distance information?")
    print(f"Response: {response1}...")
    
    time.sleep(1)  # Brief pause for demo effect
    
    # Failed workflow example
    print("\n2. Running impossible travel query (will create failed workflow)...")
    try:
        response2 = agent.run("I want to book a flight from New York to Mars for next week, what is the weather like in Mars?")
        print(f"Response: {response2}...")
    except Exception as e:
        print(f"Query completed with some failures as expected: {str(e)}...")
    
    time.sleep(1)
    
    # Another successful workflow
    print("\n3. Running currency conversion query (will create successful workflow)...")
    response3 = agent.run("What's the exchange rate from USD to EUR, and what's the weather like in Paris?")
    print(f"Response: {response3[:200]}...")
    
    # Phase 2: Show workflow retrieval
    print_section("Phase 2: Examining Created Workflows")
    
    # Retrieve workflows to show what was stored
    print("Retrieving stored workflows...")
    workflows = Workflow.retrieve_workflows_by_query("travel planning", memory_provider)
    
    print(f"\nFound {len(workflows)} workflows related to travel planning:")
    for i, workflow in enumerate(workflows, 1):
        print(f"\n--- Workflow {i} ---")
        print_workflow_info(workflow)
    
    # Phase 3: Demonstrate workflow memory in action
    print_section("Phase 3: Workflow Memory Guidance in Action")
    
    print("Now running a similar query to see how workflow memory guides execution...")
    print("The agent should reference previous successful workflows for travel planning.\n")
    
    # This query should benefit from previous workflow memory
    response4 = agent.run("I'm planning another trip from London to Tokyo. Can you help me with the same kind of information as before?")
    print(f"Response: {response4[:300]}...")
    
    print("\n" + "="*60)
    print("Notice how the agent can reference previous workflow executions")
    print("to understand what 'same kind of information' means and follow")
    print("similar successful patterns from past workflows.")
    print("="*60)
    
    # Phase 4: Show workflow failure learning
    print_section("Phase 4: Learning from Failures")
    
    print("Running another impossible travel query to show failure learning...")
    try:
        response5 = agent.run("Can you book me a flight to Mars? I know it didn't work before.")
        print(f"Response: {response5[:200]}...")
    except Exception as e:
        print(f"Query completed: {str(e)[:100]}...")
    
    print("\nThe agent should learn from previous failures and potentially")
    print("avoid repeating the same failed approaches.")
    
    # Final summary
    print_section("Workflow Memory Benefits Demonstrated")
    print("✓ Tool executions are tracked as workflows")
    print("✓ Successful workflows guide future similar tasks")
    print("✓ Failed workflows help avoid repeating mistakes")
    print("✓ Context from past executions improves task understanding")
    print("✓ Learning accumulates over time for better performance")
    
    print(f"\nAgent ID for future reference: {agent.agent_id}")
    print("You can load this agent later to see accumulated workflow memory.")



In [ ]:
try:
    demonstrate_workflow_memory()
except KeyboardInterrupt:
    print("\nDemo interrupted by user.")
except Exception as e:
    print(f"\nDemo error: {e}")
    import traceback
    traceback.print_exc()